In [20]:
import os
os.environ["CUDA_DEVICE_OREDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dataset
#from torchsummary import summary
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import random
print(torch.__version__, torchvision.__version__)
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)

1.8.0 0.2.2
TITAN RTX
11.1


In [21]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
    
max_epochs = 30

In [22]:
# randomness 제어 
# https://hoya012.github.io/blog/reproducible_pytorch/
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)

In [23]:
# dataset load

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
dataset_path = "/media/2/Network/Imagenet_dup/"
# imagenet data load
train_dataset = dataset.ImageFolder(root=dataset_path+"train",
                                       transform=transform)
val_dataset = dataset.ImageFolder(root=dataset_path+"val",
                                       transform=transform)
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                        batch_size=16,
                                        shuffle=False,
                                        num_workers=0)
val_dataloader = torch.utils.data.DataLoader(val_dataset,
                                        batch_size=64,
                                        shuffle=False,
                                        num_workers=0)
print(len(train_dataloader),len(val_dataloader))

80073 782


In [24]:
# model 로드 
vgg16_bn = torchvision.models.vgg16_bn(pretrained=True)

#if torch.cuda.device_count() > 1:
#    vgg16_bn = nn.DataParallel(vgg16_bn)

In [19]:
# only conv5_1 is trainable
ct = 0
layer_level = 1
back_layer = []
for layer in vgg16_bn.children():
#    print(ct,layer)
    if layer_level is 1:
        for layer_1 in layer:    
            print("layer 1_",ct,layer_1)
            for i,param in enumerate(layer_1.parameters()):
                if ct != 34:
                    param.requires_grad = False
                    #print(ct,"layer false")
                else :
                    if i == 0:
                        param.requires_grad = True
                        print("=====",param.shape)
                        print("%d-%d_layer true"%(ct,i))
                    else :
                        param.requires_grad = False
                        print("=====",param.shape)
                        print("%d-%d_layer false"%(ct,i))
                        
            ct +=1
    elif layer_level is 2:
    #    print("layer 2",layer)  
        for param in layer_1.parameters():
            param.requires_grad = False
            #print(ct,"layer false")
        ct +=1
    elif layer_level is 3:
        for layer_1 in layer: 
    #        print("layer 3",layer_1) 
            for param in layer_1.parameters():
                param.requires_grad = False
                #print(ct,"layer false")
            ct+=1
    layer_level +=1
    

layer 1_ 0 Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layer 1_ 1 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
layer 1_ 2 ReLU(inplace=True)
layer 1_ 3 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layer 1_ 4 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
layer 1_ 5 ReLU(inplace=True)
layer 1_ 6 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
layer 1_ 7 Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layer 1_ 8 BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
layer 1_ 9 ReLU(inplace=True)
layer 1_ 10 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layer 1_ 11 BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
layer 1_ 12 ReLU(inplace=True)
layer 1_ 13 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
layer 1_ 14 Conv2d(128, 256,

In [9]:
#criterion = torch.nn.CrossEntropyLoss(size_average = None).to(device)
criterion = torch.nn.CrossEntropyLoss(size_average = None).cuda()
optimizer = torch.optim.SGD(vgg16_bn.parameters(),lr=0.005)
lr_sche = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)


In [ ]:

vgg16.to(device)
for epoch in range(max_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader):
        inputs,labels = data
        inputs,labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        y_pred = vgg16_bn(inputs)
        # compute loss 
        loss = criterion(y_pred,labels)
        #print(epoch,i, loss.item())
        
        if not torch.isfinite(loss):
            print("WARNING: non-finite loss, ending training")
            exit(1)
        else :
            running_loss += loss.item()
        loss.backward()
        optimizer.step()
        if i % 30 == 29: 
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/30)) 
            running_loss = 0.0

In [ ]:
# evaluation phase
criterion = torch.nn.CrossEntropyLoss(size_average = None).to(device)

vgg16_bn.cuda()
vgg16_bn.eval()
total = 0
correct =0
with torch.no_grad():
    print("eval start")
    for i, data in enumerate(val_dataloader):
        inputs,labels = data
        inputs,labels = inputs.to(device), labels.to(device)
        
        y_hat = vgg16_bn(inputs)
        _, predicted = torch.max(y_hat.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if(i%50 == 49):
            print("step : {} / {}".format(i + 1, int(len(val_dataset)/labels.size(0))))
            #print(".",end="")
    print("")
print("acc of vgg16 on imagenet : %.4f %%" %(100*correct/total))    

eval start
step : 100 / 781
step : 200 / 781
